In [10]:
import pandas as pd
import sys
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,roc_auc_score,classification_report,recall_score,precision_score
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import KFold
import json
from joblib import dump
from sklearn.model_selection import cross_val_score
from collections import Counter
import  matplotlib.pyplot as plt
import os

In [ ]:
data = pd.read_csv('./Data/2D_t2/all.csv')
data["Label"] = data['Label'].apply(lambda x: x-1)
data["Name"] = data['Name'].str.replace(r'^IU_', 'IUC_', regex=True)
# data["Name"] = data['Name'].str.lower()

Apply T4: train = [AHN EMC IU MCA MCF NU], test=[NYU]

In [12]:
def get_results(acc_list,auc_list,recall_list,precision_list):    
    acc_list = np.array(acc_list)
    auc_list = np.array(auc_list)
    recall_list = np.array(recall_list)
    precision_list = np.array(precision_list)
    print(f'Recall, Average:{recall_list.mean():.4f}, Std:{recall_list.std():.4f}')
    print(f'precision, Average:{precision_list.mean():.4f}, Std:{precision_list.std():.4f}')
    print(f'Accuracy, Average:{acc_list.mean():.4f}, Std:{acc_list.std():.4f}')
    print(f'AUC, Average:{auc_list.mean():.4f}, Std:{auc_list.std():.4f}')
    return auc_list.mean()

def get_tr_vl_ts_list(dataset_dtl,fold=0):

    with open(dataset_dtl, 'r') as f:
        fold_data = json.load(f)
    test_list = []
    for name in fold_data['test_files']:
        test_list.append(name.split('.nii.gz')[0])
    # test_list = [n.lower() for n in test_list]

    train_list =[]
    for name in fold_data['cross_validation'][fold]['train_files']:
        train_list.append(name.split('.nii.gz')[0])
    # train_list = [n.lower() for n in train_list]


    val_list=[]
    for name in fold_data['cross_validation'][fold]['validation_files']:
        val_list.append(name.split('.nii.gz')[0])
    # val_list = [n.lower() for n in val_list]
    
    return train_list,val_list,test_list

In [13]:
train_test_info = 'Train_Test_1'

In [14]:
dataset_dtl_path = f'/home/pyq6817/IPMN-Radiomics-Plus-Deeplearning/{train_test_info}.json'
train_list,val_list,test_list = get_tr_vl_ts_list(dataset_dtl=dataset_dtl_path,fold=0)

In [15]:
# train_data = data[data['Name'].isin(train_list)]
test_data = data[data['Name'].isin(test_list)]
# val_data = data[data['Name'].isin(val_list)]
# For grid search
train_val_data = data[data['Name'].isin(val_list+train_list)]

## 2D Features

In [16]:
# # feature_list.sort()
# # Test 4
# feature_list = [
# 'skewness-Laws R5S5',
# 'Collage_kurt_MaximalCorrelationCoefficient_1_nb_4_ws_5',
# 'Collage_skew_InformationMeasureOfCorrelation2_1_nb_8_ws_3',
# 'Collage_kurt_Contrast_1_nb_8_ws_7',
# 'Collage_median_MaximalCorrelationCoefficient_1_nb_32_ws_3',
# 'Collage_skew_Correlation_1_nb_16_ws_5',
# 'Collage_skew_MaximalCorrelationCoefficient_1_nb_4_ws_3',
# 'Collage_skew_SumVariance_1_nb_8_ws_5',
# 'median-Laws E5L5',
# 'Collage_skew_Entropy_1_nb_16_ws_7',
# 'skewness-Laws S5E5',
# 'skewness-Laws W5L5',
# 'Collage_skew_DifferenceEntropy_1_nb_4_ws_3',
# 'Collage_var_Contrast_1_nb_8_ws_7',
# 'Collage_kurt_SumEntropy_1_nb_16_ws_5',
# ]

# # test3
# # feature_list = ['Collage_skew_InformationMeasureOfCorrelation2_1_nb_8_ws_3',
# # 'skewness-Laws S5E5',
# # 'skewness-Laws R5S5',
# # 'Collage_skew_InformationMeasureOfCorrelation2_1_nb_16_ws_5',
# # 'Collage_skew_Correlation_1_nb_8_ws_5',
# # 'median-Laws E5L5',
# # 'median-Laws W5S5',
# # 'Collage_skew_MaximalCorrelationCoefficient_1_nb_4_ws_3']

# # test2
# # feature_list = ['Collage_kurt_MaximalCorrelationCoefficient_1_nb_4_ws_5',
# # 'median-Laws E5L5',
# # 'Collage_skew_InformationMeasureOfCorrelation2_1_nb_8_ws_3',
# # 'Collage_var_InformationMeasureOfCorrelation2_1_nb_16_ws_3',
# # 'skewness-Laws R5S5',
# # 'Collage_skew_MaximalCorrelationCoefficient_1_nb_4_ws_3',
# # 'skewness-Laws S5E5',
# # 'median-Laws W5S5',
# # 'Collage_kurt_MaximalCorrelationCoefficient_1_nb_4_ws_7',
# # 'Collage_median_SumEntropy_1_nb_8_ws_3',
# # 'Collage_kurt_SumEntropy_1_nb_16_ws_5',
# # 'Collage_skew_SumVariance_1_nb_4_ws_7']

# # test1
# feature_list = ['Collage_skew_InformationMeasureOfCorrelation2_1_nb_8_ws_3',
# 'Collage_var_InformationMeasureOfCorrelation2_1_nb_16_ws_3',
# 'Collage_kurt_Contrast_1_nb_16_ws_7',
# 'Collage_skew_MaximalCorrelationCoefficient_1_nb_4_ws_3',
# 'Collage_kurt_SumAverage_1_nb_16_ws_3',
# 'Collage_kurt_InformationMeasureOfCorrelation2_1_nb_16_ws_3',
# 'Collage_kurt_MaximalCorrelationCoefficient_1_nb_4_ws_5',
# 'median-Laws E5L5',
# 'Collage_kurt_SumEntropy_1_nb_16_ws_3',
# 'skewness-Laws S5E5',
# 'kurtosis-Haralick correlation ws=7 n=4',
# 'Collage_kurt_InformationMeasureOfCorrelation2_1_nb_64_ws_5',
# 'Collage_median_DifferenceVariance_1_nb_64_ws_7',
# 'skewness-Laws E5S5',
# 'skewness-Gradient sobelxy']

## 3D Features

In [ ]:
train_test_info = 'Train_test_1'
if train_test_info =='Train_Test_1':
        # Test 4
        feature_list = [
'Collage_var_InformationMeasureOfCorrelation2_1_nb_8_ws_5',
'Collage_var_SumEntropy_1_nb_32_ws_7',
'skewness-Haralick info1 ws=5 n=4',
'skewness-Laws  S5L5E5',
'median-Laws  E5E5E5',
'Collage_skew_DifferenceEntropy_1_nb_32_ws_7',
'median-Laws  R5E5S5',
'median-Laws  W5E5S5',
'Collage_kurt_InformationMeasureOfCorrelation2_1_nb_4_ws_5',
'median-Laws  E5L5S5',
        ]
    elif train_test_info =='Train_Test_2':
        # test3
        feature_list = ['Collage_var_InformationMeasureOfCorrelation2_1_nb_16_ws_3',
'median-Laws  E5L5S5',
'median-Laws  E5E5E5',
'skewness-Laws  R5R5L5',
'median-Laws  L3S3L3',
'skewness-Laws  S3E3S3',
'Collage_kurt_Correlation_2_nb_64_ws_5',
'Collage_kurt_SumAverage_1_nb_32_ws_7',
'skewness-Laws  S5L5E5',
'skewness-Laws  S3E3E3',
'skewness-Laws  W5L5S5',
'kurtosis-Laws  L5R5L5',]
    elif train_test_info =='Train_Test_3':
        # test2
        feature_list = ['skewness-Laws  S5W5R5',
'skewness-Laws  S5L5E5',
'median-Laws  L3S3L3',
'median-Laws  S3E3E3',
'Collage_skew_MaximalCorrelationCoefficient_1_nb_4_ws_5',
'median-Laws  R5E5S5',
'skewness-Laws  W5L5S5',
'Collage_kurt_SumAverage_1_nb_32_ws_7',
'Collage_var_InformationMeasureOfCorrelation2_1_nb_16_ws_3',
'median-Laws  E5L5S5',
'Collage_kurt_InformationMeasureOfCorrelation2_1_nb_4_ws_5',
'Collage_kurt_InformationMeasureOfCorrelation2_2_nb_16_ws_7',
'Collage_kurt_InformationMeasureOfCorrelation2_1_nb_16_ws_3',
'median-Laws  E5E5E5',
'median-Laws  W5E5S5',]
    elif train_test_info =='Train_Test_4':
        feature_list = ['skewness-Laws  E5E5R5',
'skewness-Laws  E3S3E3',
'skewness-Laws  R5E5E5',
'skewness-Haralick correlation ws=5 n=4',
'skewness-Laws  E3E3E3',
'Collage_kurt_SumAverage_1_nb_16_ws_7',
'Collage_kurt_MaximalCorrelationCoefficient_1_nb_4_ws_7',
'skewness-Haralick info1 ws=5 n=64',
'Collage_kurt_InformationMeasureOfCorrelation2_2_nb_16_ws_7',
'skewness-Laws  R5R5L5',
'Collage_var_SumEntropy_1_nb_32_ws_7',
'median-Laws  R5E5L5']

In [17]:
X = train_val_data.drop(columns=['Center','Name','Label'])
y = train_val_data[["Label"]]
standard_scaler = StandardScaler()
X_scaled = pd.DataFrame(standard_scaler.fit_transform(X),columns=X.columns)
X_selected = X_scaled[feature_list]

param_grid = {
    'n_estimators': [40,43,45,47],          
    'max_depth': [8,9,10,11],            
    'min_samples_split': [5],         
    'min_samples_leaf': [1,2,5],           
    'max_features': [None],  
    'bootstrap': [True]               
} 

# GridSearchCV get bset hyper parameters
rf_model = RandomForestClassifier(random_state=3)
grid_search = GridSearchCV(
    estimator=rf_model,           # 目标模型
    param_grid=param_grid,        # 参数网格
    cv=5,                         # 5折交叉验证
    scoring='roc_auc',           # 评分标准（可以选择'accuracy', 'f1'等）
    n_jobs=-1,                    # 使用所有可用内核
    verbose=1                     # 输出搜索过程
)


grid_search.fit(X_selected, y)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/data2/pyq6817/.conda/envs/medical-image/lib/python3.10/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/data2/pyq6817/.conda/envs/medical-image/lib/python3.10/site-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/data2/pyq6817/.conda/envs/medical-image/lib/python3.10/site-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/data2/pyq6817/.conda/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/data2/pyq6817/.conda/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:14

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=3), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [8, 9, 10, 11],
                         'max_features': [None], 'min_samples_leaf': [1, 2, 5],
                         'min_samples_split': [5],
                         'n_estimators': [40, 43, 45, 47]},
             scoring='roc_auc', verbose=1)

In [18]:
results = grid_search.cv_results_
for i in range(len(results['params'])):
    print(f"Search {i + 1}: {results['params'][i]}")
    mean_test_auc = results['mean_test_score'][i]
    print(f"Average AUC: {mean_test_auc:.4f}")
    
    for fold in range(grid_search.cv):
        # fold_test_accuracy = results[f'split{fold}_test_score'][i]
        fold_test_auc = results[f'split{fold}_test_score'][i]
        print(f"  Fold {fold + 1} - , validation AUC: {fold_test_auc:.4f}")
    print("-" * 40)

Search 1: {'bootstrap': True, 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 40}
Average AUC: 0.7437
  Fold 1 - , validation AUC: 0.7096
  Fold 2 - , validation AUC: 0.6548
  Fold 3 - , validation AUC: 0.7827
  Fold 4 - , validation AUC: 0.7904
  Fold 5 - , validation AUC: 0.7811
----------------------------------------
Search 2: {'bootstrap': True, 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 43}
Average AUC: 0.7414
  Fold 1 - , validation AUC: 0.7115
  Fold 2 - , validation AUC: 0.6529
  Fold 3 - , validation AUC: 0.7740
  Fold 4 - , validation AUC: 0.7885
  Fold 5 - , validation AUC: 0.7801
----------------------------------------
Search 3: {'bootstrap': True, 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 45}
Average AUC: 0.7449
  Fold 1 - , validation AUC: 0.7192
  Fold 2 - , validation AUC: 0.6548
  Fold 3 - , validati

Radiomics Test data

In [19]:
x_test_raw = test_data.drop(columns=['Center','Name','Label'])
y_test = test_data[["Label"]]
x_test_scaled = pd.DataFrame(standard_scaler.transform(x_test_raw),columns=x_test_raw.columns)
x_test = x_test_scaled[feature_list]

In [20]:
models = []
test_acc_list = []
test_auc_list = []
test_recall_list = []
test_precision_list = []
val_acc_list = []
val_auc_list = []
val_recall_list = []
val_precision_list = []

kf = KFold(n_splits=5, shuffle=True, random_state=6)
for fold in range(5):
    train_list,val_list,test_list = get_tr_vl_ts_list(dataset_dtl=dataset_dtl_path,fold=fold)
    train_data = data[data['Name'].isin(train_list)]
    val_data = data[data['Name'].isin(val_list)]

    x_train_raw = train_data.drop(columns=['Center','Name','Label'])
    y_train = train_data[["Label"]]
    x_train_scaled = pd.DataFrame(standard_scaler.transform(x_train_raw),columns=x_train_raw.columns)
    x_train = x_train_scaled[feature_list]

    x_val_raw = val_data.drop(columns=['Center','Name','Label'])
    y_val = val_data[["Label"]]
    x_val_scaled = pd.DataFrame(standard_scaler.transform(x_val_raw),columns=x_val_raw.columns)
    x_val = x_val_scaled[feature_list]
    
    # Get best hyper params
    best_params = grid_search.best_params_
    best_rf_model = RandomForestClassifier(**best_params, random_state=42)
    best_rf_model.fit(x_train, y_train)
    # Save model
    models.append(best_rf_model)
    
    fold_accuracy = accuracy_score(y_val, best_rf_model.predict(x_val))
    val_acc_list.append(fold_accuracy)
    fold_auc = roc_auc_score(y_val, best_rf_model.predict_proba(x_val)[:, 1])
    val_auc_list.append(fold_auc)
    print(f"Fold {fold + 1} - ACC: {fold_accuracy:.4f}, AUC: {fold_auc:.4f}")
    test_accuracy = accuracy_score(y_test, best_rf_model.predict(x_test))
    test_acc_list.append(test_accuracy)
    test_auc = roc_auc_score(y_test, best_rf_model.predict_proba(x_test)[:, 1])
    test_auc_list.append(test_auc)
    print(f"Test {fold + 1} - ACC: {test_accuracy:.4f}, AUC: {test_auc:.4f}")
    print("-" * 40)
print('Validation set')
get_results(val_acc_list,val_auc_list,val_recall_list,val_precision_list)
print("Test set")
get_results(test_acc_list,test_auc_list,test_recall_list,test_precision_list)
print('================================================')


/data2/pyq6817/.conda/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/data2/pyq6817/.conda/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Fold 1 - ACC: 0.7015, AUC: 0.7352
Test 1 - ACC: 0.4483, AUC: 0.4216
----------------------------------------
Fold 2 - ACC: 0.6615, AUC: 0.6902
Test 2 - ACC: 0.4828, AUC: 0.4608
----------------------------------------


/data2/pyq6817/.conda/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/data2/pyq6817/.conda/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Fold 3 - ACC: 0.6818, AUC: 0.7288
Test 3 - ACC: 0.4828, AUC: 0.4853
----------------------------------------
Fold 4 - ACC: 0.6970, AUC: 0.7190
Test 4 - ACC: 0.5517, AUC: 0.4902
----------------------------------------
Fold 5 - ACC: 0.6923, AUC: 0.7754
Test 5 - ACC: 0.4828, AUC: 0.4559
----------------------------------------
Validation set
Recall, Average:nan, Std:nan
precision, Average:nan, Std:nan
Accuracy, Average:0.6868, Std:0.0142
AUC, Average:0.7297, Std:0.0276
Test set
Recall, Average:nan, Std:nan
precision, Average:nan, Std:nan
Accuracy, Average:0.4897, Std:0.0338
AUC, Average:0.4627, Std:0.0245


/data2/pyq6817/.conda/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/tmp/ipykernel_189456/697513518.py:6: RuntimeWarning: Mean of empty slice.
  print(f'Recall, Average:{recall_list.mean():.4f}, Std:{recall_list.std():.4f}')
/data2/pyq6817/.conda/envs/medical-image/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/data2/pyq6817/.conda/envs/medical-image/lib/python3.10/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/data2/pyq6817/.conda/envs/medical-image/lib/python3.10/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in div

Save model, since models are already saved, their is no need to 

In [41]:
# root = './trained_models'
# save_path = os.path.join(root,train_test_info)
# if not os.path.exists(save_path):
#     os.mkdir(save_path)
# for index,model in enumerate(models):
#     dump(model, os.path.join(save_path,f'{train_test_info}_Fold{index}.joblib'))